# Sentiment analýza

V tomto notebooku se budeme věnovat analýze textu - konkrétně recenzím z csfd.cz. Naším cílem bude odhadnout z textu uživatelova literárního počinu, kolik hvězdiček určitému filmu dal.
Jaké balíčky použijeme? S daty budeme pracovat v pandích datasetech, tudíž Pandas bude nepostradatelný. První dataset je uložen v xml souboru, který nějak do pand musíme dostat - proto ElementTree. Druhý dataset si nascrapujeme přímo z webu. Jelikož to se bude odehrávat v separátním skriptu a nikoli v notebooku, balíček, který nám tuto operaci umožní, se v následující importovací buňce nenalézá. Nicméně pokud si to budete chtít zkusit, musíte si nainstalovat *scrapy*. 
Pro konverzi textu na vektory, které jsou pro počítač přeci jen přirozenější, jakožto i pro výpočet modelů nad oněmi vektory, použijeme balíček scikit-learn. Předtím bude ale třeba slova v recenzních lemmatizovat - na to se použije Morphodita. Ostatní importované balíčky jsou použity už jenom minoritně.

In [1]:
import pandas as pd
import xml.etree.ElementTree as etree
import string

import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

import ufal.morphodita as ufm
morpho = ufm.Morpho.load("czech-morfflex-161115.dict")

import re
import collections
import random

## Dataset s obecnými recenzemi

"Obecné" recenze pocházejí z datasetu dostupného na http://liks.fav.zcu.cz/sentiment/. Jsou uloženy v xml-ku, tudíž si je nejdříve musíme převést na několik listů a z těch si pak vytvořit dataframe.

In [5]:
tree = etree.parse("csfd-90k-reviews-ranlp2013.xml")
root = tree.getroot()
reviews = root.findall(".//review")

name = []
rating = []
text = []
for review in reviews:
    rating.append(review.attrib["origRating"])
    name.append(review.attrib["product"])
    text.append(review.find("./text").text)

general_reviews = pd.DataFrame({"name":name, "rating":rating, "text":text})

Podívejme se, jak vlastně dataset vypadá. Zdálo by se, že ve sloupci rating jsou integery. To zrovna nechceme - mezi jednotlivými známkami neplánujeme definovat sčítání anebo odečítání. Nicméně funkce info nám prozradí, že Python je zjevně přečetl jako stringy. K jejich seznamu se dostaneme skrze funkci unique. Další analýzou se dá ukázat, že '0' odpovídá hodnocení "Odpad" a -1 značí "Nehodnoceno".

In [6]:
general_reviews.head()

,name,rating,text
0,Mein langsames Leben (2001),5,Druhý film Angely Schanelec o počasí. :) Po zh...
1,Mein langsames Leben (2001),2,U Pomalého života jsem strávil 80 minut a teď ...
2,Můj otec / Mein Vater (TV film) (2003),2,"Nevím, jestli je to nepovedeným dabingem, otra..."
3,Můj otec / Mein Vater (TV film) (2003),5,"Upřímný a velice smutný film, chvílemi tak dep..."
4,Můj otec / Mein Vater (TV film) (2003),2,"Možná to bude trochu divný komentář, ale nemůž..."


In [7]:
general_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92398 entries, 0 to 92397
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    92398 non-null  object
 1   rating  92398 non-null  object
 2   text    92398 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [8]:
general_reviews["rating"].unique()

array(['5', '2', '0', '4', '1', '-1', '3'], dtype=object)

In [9]:
zero_rating_amount = len(general_reviews[general_reviews["rating"]=='0'])
minus_one_rating_amount = len(general_reviews[general_reviews["rating"]=='-1'])
print(f"Amount of zero rating: {zero_rating_amount}")
print(f"Amount of minus one rating: {minus_one_rating_amount}")

Amount of zero rating: 29808
Amount of minus one rating: 6


In [10]:
general_reviews[general_reviews["rating"]=='-1']

,name,rating,text
1517,Torontské stíny / Blue Murder (TV seriál) (2001),-1,Jen další z řad kriminálních příběhů...
31536,LazyTown / LazyTown (TV seriál) (2004),-1,Nehodnotitelné.
33941,Zákon a pořádek: Zločinné úmysly / Law & Order...,-1,Jen další z řad kriminálních příběhů...
37983,Můj přítel Monk / Monk (TV seriál) (2002),-1,"Další krimi seriál, kterých je už víc než dost..."
46754,"Vy nám taky, šéfe! (2008)",-1,HUDBA: Tomáš Polák
58192,1000 Frames (1960),-1,Nehodnotitelné.


Nyní si každou recenzi převedeme na malá písmena. Následně si recenze a jejich hodnocení rozdělíme na trénovací a testovací data. Jelikož zde chci ukázat, co se v kterém bodě děje, a chci, abyste měli i vy při svých experimentech stejné výsledky, nastavuji natvrdo random_state na určitou konstantní hodnotu.

In [11]:
gen_review_strings = [text.strip().lower() for text in general_reviews["text"]]

x_train, x_test, y_train, y_test = train_test_split(
     gen_review_strings, 
     general_reviews["rating"], 
     test_size=0.2, 
     random_state=42
)

Připravíme si pomocnou funkci, která v dalším kroku převede recenzi coby spojitý text na list tokenů - jednotlivých slov recenze. Se slovy se navíc odehraje několik dalších věcí. Je odstraněna interpunkce. Pravě proto, že někde potřebujeme vzít seznam všech možných interpunkčích znaků, jsme importovali balíček string. 
Ze seznamu slov vyhazujeme tzv. stopwords alias slova, která se vyskytují tak často, že víceméně nenesou žádnou podstatnou informaci. Popravdě zde moc stop slov nemám - snad postupně budu jejich seznam doplňovat. Na internetu se dá pravda i pro češtinu najít řada seznamů takovýchto slov. Nicméně některá z nich mi přijdou pro potřeby recenzí vcelku důležitá (např. nula, musí, dobrý atd.).
Krom toho všechno v pomocné funkci probíhá i lematizace - koneckonců podle toho se funkce i jmenuje. Co to lematizace znamená? Jedná se o převod různých tvarů téhož slova na slovo jedno. Například "já", "byl" či "budete" se všechny převedou na "být".Díky této operaci se faktický počet slov z recenzí snižuje, aniž by se redukoval význam v těchto slovech schovaný.

In [12]:
PUNCTUATIONS = string.punctuation
NUMBER_OF_PUN_CHARS = len(PUNCTUATIONS)
PUN_REPLACING_STRING = " " * NUMBER_OF_PUN_CHARS
STOPWORDS = ["a", "i", "nebo", "proto"]

def remove_punctuation(text):
    return text.translate(str.maketrans(PUNCTUATIONS, PUN_REPLACING_STRING, ""))

def lemmatize_review(review_text):
    lemmas = ufm.TaggedLemmas()
    lemmas_list = []
    review_text_no_pun = remove_punctuation(review_text)
    tokens = review_text_no_pun.split()
    filtered_tokens = [token for token in tokens if token not in STOPWORDS]
    for token in filtered_tokens:
        morpho.analyze(token, morpho.GUESSER, lemmas)
        lemmas_list.append(lemmas[0].lemma)
    return lemmas_list

Pomocnou funkci lemmatize_review využijeme jako tokenizer ve vektorizéru TfidfVectorizer. Co předchozí věta vlastně znamenala? Počítač sám o sobě se slovy moc dobře pracovat neumí, musí se mu převést na nějakou posloupnost čísel - na vektory (fakticky asi pole či něco podobného, ale to zde nemá cenu řešit). Velikost vektorů je dána počtem slov v celém konglomerátu dokomentů (zde recenzí). Samotný dokument je pak reprezentován takovouto posloupností čísel, z nichž každé je navázáno na určité slovo a udává v tom nejjednoduším případě, kolikrát se ono slovo v recenzi vyskytuje.
Uvedmě si příklad. Mějme balík dvou dokumentů: "skákal skákal pes přes oves" a "pes oves nebaští". V balíku se vyskytuje celkem pět slov, to bude tedy i délka vektoru. První číslo vektoru bude navázáno na "skákal", druhé na "pes", třetí na "přes", čtvrté na "oves" a páté na "nebaští". Vektor prvního dokumentu tedy vypadá jako [2,1,1,1,0], vektor druhého dokumentu má podobu [0,1,0,1,1].
Výše uvedenou transformaci dělá CountVectorizer. Nicméně v kódu je TfidfVectorizer. V dokumentaci byste si mohli přečíst, že de facto platí TfidfVectorizer =  CountVectorizer + TfidfTransformer. K čemu ta Tfidf věc je? Jedná se o zkratku term frequency-inverse document frequency. Jde prakticky o to, že váha určitého slova pro analýzu roste s počtem jeho výskytů v dokumentu. Na druhou stranu některá slova se vyskytují téměř v každém dokumentu a tudíž nám toho o konkrétním dokumentu mnoho neřeknou (například spojky či předložky). Bylo by tedy užitečné čísla ve vektorech nějak nanormovat, aby slova charakteristická pro několik málo dokumentů měla větší váhu než slova, která díky své vysoké frekvenci výskytu nemohou charakterizovat vůbec nic. No a to dělá právě TfidfTransformer.
Vektorizér by sám od sebe pouze oddělovat jednotlivá slova podle mezer. Můžeme mu ale předat svou vlastní funkci, která krom oddělení slov od sebe s nimi udělá ještě něco jiného - v našem případě lematizaci a odstranění interpunkce.
V parametrech TfidfVectorizer jsou kromě tokenizeru použity i další parametry: ngram_range a min_df. Parametr min_df slouží k omezení velikosti vektorů - z celkového "slovníku" vytvořeného ze všech slov všech dokumentů totiž vyhodí ta slova, která se prakticky vůbec nevyskytují (zde ta, která se objeví ve všech dokumentech dohromady maximálně čtyřikrát). Co se ngram_range týče, některá slova mají v souvislosti se slovy jinými speciální význam - např "galerie" a "Národní galerie". To druhé se nazývá bigram, shluk pravidleně se společně vyskytujících tří slov nese označení trigram. Slovo stojící samostatně je unigram. Rozsah ngramů ve tvaru (1,3) pak znamená, že se do slovníku dostanu nejen slova jednotlivá, ale i dvojice a trojice.
V prvním řádku v následující buňce se sice vytváří vektorizér, nicméně recenze se na vektory převedou až na řádkce následující. Na ní se na trénovací recenze vypouští funkce fit_transform. Tu lze pojmout jako součet funkcí fit a transform. Právě funkce fit slouží k tomu, aby se model slova "naučil" - namapoval je na určitou polohu v později vytvářených vektorech. Následně funkce transform převádí jednotlivé dokumenty-recenze na ony vektory. Všimněte si, že zatímco na trénovací data je vypuštena funkce fit_transfor, na data testovací data už působí pouze funkce transform. Důvodem je konzistence. Na to, abychom mohli později vytvořený ML model vypustit na nová testovací data, musí být slovník převádějící slova na vektory stejný jak pro trénovací, tak pro testovací záznamy. Jinými slovy převodní slovník si vytvoříme pouze jednou a poté ho aplikujeme na všechna data, která nám přidou pod ruku.

In [13]:
vectorizer = TfidfVectorizer(tokenizer = lemmatize_review, ngram_range=(1,3), min_df=5) 
vectorized_train = vectorizer.fit_transform(x_train)
vectorized_test = vectorizer.transform(x_test)

Model, který použijeme k odhadu počtu hvězdiček, patří mezi ty nejprimitivnější - jde o logistickou regresi. Při inicializaci modelu mu říkáme, že má použít solver saga. Solver je matematický algoritmus, který se snaží nastavit parametry našeho modelu tak, aby v kombinaci se zdrojovými daty co možná nejlépe sedly na očekávané výsledné hodnoty. Problém s defaultním solverem je v tom, že se pro velké matice se zdrojovými daty zadýchává a nechce moc konvergovat. Oproti tomu saga solver by měl být pro velké matice (ideálně skládající se z valné části z nul) ideální. V našem případě vedou oba solvery ke zhroba stejné hodnotě přesnosti, avšak saga je hotová za 20s, zatímco defaultní solver skončí po 35 sekundách a navích s warning hláškou.

In [14]:
logist_model = LogisticRegression(solver="saga")
logist_model.fit(vectorized_train, y_train)

prediction = logist_model.predict(vectorized_test)

accuracy = sklearn.metrics.accuracy_score(y_test, prediction)
accuracy

0.8103354978354979

In [15]:
comparison = pd.DataFrame({"predikce":prediction, "realita":y_test, "text":x_test})
comparison.head()

,predikce,realita,text
38331,5,5,mám za sebou k dnešnímu dni celkem 5 projekcí....
72181,2,0,není to zklamání de nira. na vině byl tentokrá...
20691,0,2,"z transek atd. je mi na blití. což o to, otrlý..."
32674,0,0,29.8.2007 11:00 - právě běží na nově tento fil...
72135,0,0,pcha! a pak ze nejde natocit opravdovou zmrdov...


Pro rozumnější srovnání bude lepší použít confusion matici. To je tabulka čísel, kde ve sloupcích jsou ukázány reálná hodnocení, zatímco na řádcích jsou predikovaná hodnocení. V matici v následující buňce odpovídá první řádek/sloupec hodnocení "odpad", druhý řádek/sloupec jedné hvězdičce, poslední řádek/sloupec pěti hvězdičkám. Číslo na prvním řádku prvního sloupce nám tedy prozrazuje, že u 4710 recenzí bylo správně predikováno hodnocení odpad. Cifra na třetím řádku a prvním sloupci zase říká, že 964 recenzí mělo sice fakticky hodnocení odpad, ale podle predikce měly tyto recenze mít dvě hvězdičky. Ideální situace by u konfidenční matice byla taková, kdyby všechna čísla ležela v její stopě (tj. v diagonále jdoucí z levého horního rohu do pravého dolního rohu).

In [16]:
conf_matrix = sklearn.metrics.confusion_matrix(y_test, prediction)
conf_matrix

array([[4709,    0,  929,    0,    0,  316],
       [   6,    0,    0,    0,    0,    0],
       [ 964,    0, 4742,    2,    0,  496],
       [   1,    0,    4,    9,    0,    0],
       [   0,    0,    0,    0,    3,   15],
       [ 309,    0,  458,    0,    5, 5512]], dtype=int64)

Vytvořili jsme sice predikční model, ale je lepší než náhodné hádání? To zjistíme tak, že si toto náhodné hádání nasimulujeme. Nejprve se podíváme, jaký je v původních datech (před dělením na trénovací a testovací) počet jednotlivých typů hodnocení.

In [17]:
ratings_freq = collections.Counter(general_reviews["rating"])
ratings_freq

Counter({'5': 31468,
         '2': 30902,
         '0': 29808,
         '4': 102,
         '1': 34,
         '-1': 6,
         '3': 78})

Následně si vytvoříme list skládající se z odpovídajícího počtu jednotlivých hodnocení. Ten promícháme s pomocí funkce random.shuffle a vybereme z něj prvních X elementů, kde X je rovno počtu testovacích recenzí.

In [18]:
random_with_real_freq = ['-1']*6 + ['0']*29808 + ['1']*34 + ['2']*30902 + ['3']*78 + ['4']*102 + ['5']*31468
random.shuffle(random_with_real_freq)
fake_prediction = random_with_real_freq[:len(y_test)]

Nyní můžeme jednak spočítat "náhodnou" accuracy, jednak si zobrazit "náhodnou" confusion matrix. Je patrné, že přesnost modelu je mnohem lepší než náhodné tipování.

In [20]:
print(sklearn.metrics.accuracy_score(y_test, fake_prediction))
conf_matrix = sklearn.metrics.confusion_matrix(y_test, fake_prediction)
conf_matrix

0.33755411255411255


array([[   0,    0,    0,    0,    0,    0,    0],
       [   0, 1972,    3, 1930,    7,   10, 2032],
       [   0,    2,    0,    1,    0,    0,    3],
       [   0, 2009,    4, 2132,    6,    8, 2045],
       [   0,    5,    0,    5,    0,    0,    4],
       [   0,    8,    0,    6,    0,    0,    4],
       [   1, 2073,    5, 2063,    6,    2, 2134]], dtype=int64)

Vytvořme si nyní dataframe ukazující, jaký vliv má přítomnost určitého slova na umístění recenze do určité třídy. 
Tento dataframe vytvoříme pomocí slovníku vektorizéru a koeficientů z logistické regrese pro jednotlivé třídy (každá třída reprezentuje jiný počet hvězd). Musíme však zajistit, aby byly slova a jejich koeficienty na sebe správně navázány.
Slovník vektorizéru má podobu slovníku (překvapivě...), kde klíčem je slovo a hodnotou jeho index. Tento slovník - vlastně slovníky obecně - ale nejsou uspořádány a to je problém. Koeficienty totiž uspořádány jsou a to právě podle indexu slova.  

In [21]:
vectorizer.vocabulary_

{'zapomenutelný_^(*6out)': 136385,
 'film': 29645,
 'belmondo_;S': 7547,
 'být': 9691,
 'totiž-1': 118017,
 'pan_^(oslovení)': 77324,
 'herec': 33452,
 'obvzlášť-1': 72835,
 'pobavit_:W': 79236,
 'scéna': 95236,
 's-1': 93518,
 'ferrari_^(vozidlo)': 29517,
 'jeho_^(přivlast.)': 41319,
 'replika': 91212,
 'já': 44252,
 'ženská_^(osoba)': 145489,
 'no': 68595,
 'fuj': 32431,
 'zapomenutelný_^(*6out) film': 136391,
 'belmondo_;S být': 7548,
 'být totiž-1': 15141,
 'pan_^(oslovení) herec': 77335,
 'scéna s-1': 95424,
 'já ženská_^(osoba)': 46070,
 'ženská_^(osoba) no': 145505,
 'no fuj': 68642,
 'chytnout_:W': 18743,
 'ten': 111197,
 'už-1': 122277,
 'několikrát': 71086,
 'muset': 59314,
 'říci': 142908,
 'že-1': 144147,
 'tenhle': 115840,
 'opravdu-1': 75455,
 'hodný-1_^(být_hoden)': 35552,
 'přesto-1': 88745,
 'ala-1_,t_^(místnost_v_starořím._obydlí;;vojenská_jednotka_ve_st._Římě;;boční_loď_v_bazilice)': 2790,
 'nejeden': 66780,
 'on-1': 74517,
 'rozesmát': 92425,
 'rozesmívat_:T': 92433

Musíme tedy slovník jednak převést na list, jednak v tomto listu uspořádat podle hodnoty hodnoty.

In [22]:
sorted(vectorizer.vocabulary_, key=lambda x: vectorizer.vocabulary_[x])

['0',
 '0 0',
 '0 10',
 '0 10 hudba',
 '0 5',
 '0 5 10',
 '0 být',
 '0 jeden`1',
 '0 odpad',
 '0 ten',
 '00',
 '000',
 '000 000',
 '000 dolar_;b',
 '000 člověk',
 '007',
 '01',
 '02',
 '03',
 '03 2009',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '09 2010',
 '1',
 '1 0',
 '1 1',
 '1 1 2',
 '1 10',
 '1 10 dít-1_^(dít_se)',
 '1 10 hudba',
 '1 2',
 '1 2 3',
 '1 2 být',
 '1 2 díl_^(jeden_díl_[z_celku])',
 '1 2 film',
 '1 2011',
 '1 3',
 '1 4',
 '1 5',
 '1 5 5',
 '1 5 hodina_^(jednotka_času)',
 '1 5 hvězdička',
 '1 8',
 '1 85',
 '1 85 1',
 '1 ala-1_,t_^(místnost_v_starořím._obydlí;;vojenská_jednotka_ve_st._Římě;;boční_loď_v_bazilice)',
 '1 bod',
 '1 být',
 '1 být ten',
 '1 dobrý',
 '1 díl_^(jeden_díl_[z_celku])',
 '1 díl_^(jeden_díl_[z_celku]) být',
 '1 film',
 '1 hodina_^(jednotka_času)',
 '1 hvězda',
 '1 hvězda za',
 '1 hvězdička',
 '1 hvězdička za',
 '1 když',
 '1 končit_:T',
 '1 končit_:T gemblér',
 '1 který',
 '1 minout',
 '1 místo-1_^(fyzické_umístění)',
 '1 na-1',
 '1 oblíbený_^(*3it)

Nyní již vytvoření dataframu nic nebrání. Koeficienty bereme z model2.coef_, které je listem listů, až od indexu 1 - nula se týká extrémně řídce zastoupené třídy "Nehodnoceno". Sloupeček "goodness" si definujeme jako míru, zda má slovo při binárním rozhodování odpad/5 hvězd kladné či záporné znaménko. 

In [23]:
word_weight = pd.DataFrame({
    "word":sorted(vectorizer.vocabulary_, key=lambda x: vectorizer.vocabulary_[x]),
    "class_0": logist_model.coef_[1],
    "class_1": logist_model.coef_[2],
    "class_2": logist_model.coef_[3],
    "class_3": logist_model.coef_[4],
    "class_4": logist_model.coef_[5],
    "class_5":logist_model.coef_[6],
    "goodness": logist_model.coef_[6]-logist_model.coef_[1]
})
word_weight

,word,class_0,class_1,class_2,class_3,class_4,class_5,goodness
0,0,6.960406,-0.020337,-3.989952,-0.025558,-0.035268,-2.882916,-9.843322
1,0 0,-0.477822,-0.000366,0.307347,-0.000474,-0.000645,0.172079,0.649901
2,0 10,1.731517,-0.004001,-0.918591,-0.005254,-0.007337,-0.795019,-2.526536
3,0 10 hudba,0.140245,-0.000121,-0.107248,-0.000148,-0.000206,-0.032479,-0.172724
4,0 5,0.543323,-0.001677,-0.169796,-0.002073,-0.002957,-0.366292,-0.909615
...,...,...,...,...,...,...,...,...
146105,♥,-0.062849,-0.000572,-0.181667,-0.000806,-0.002200,0.248276,0.311124
146106,♥♥♥,-0.407929,-0.000844,-0.310753,-0.001123,-0.001502,0.722428,1.130358
146107,♫,-0.422813,-0.001498,-0.309662,-0.002091,-0.002847,0.739364,1.162177
146108,♫ hodnocený_^(*4tit),-0.049663,-0.000202,-0.051176,-0.000273,-0.000357,0.101742,0.151405


Proveďme sanity check - podívejme se na slova a sousloví obsahující slovo "hnusný". Zdá se, že všechno funguje - většinově zde vidíme příspěvky do odpadu.

In [24]:
word_weight[word_weight["word"].str.contains("hnusný")]

,word,class_0,class_1,class_2,class_3,class_4,class_5,goodness
10704,být fakt-1 hnusný,0.013194,-0.000142,0.022074,-0.000181,-0.000240,-0.034658,-0.047852
10864,být hnusný,0.444185,-0.001608,-0.256437,-0.002095,-0.002818,-0.180698,-0.624882
14305,být taka-1_;L hnusný,-0.120708,-0.000305,0.063814,-0.000368,-0.000492,0.058165,0.178873
14609,být ten hnusný,0.040619,-0.000437,0.089274,-0.000557,-0.000727,-0.128026,-0.168644
29156,fakt-1 hnusný,0.013194,-0.000142,0.022074,-0.000181,-0.000240,-0.034658,-0.047852
34882,hnusný,2.127405,-0.015542,-0.696770,-0.019683,-0.027189,-1.363341,-3.490746
34883,"hnusný ala-1_,t_^(místnost_v_starořím._obydlí;...",-0.240470,-0.000557,0.242038,-0.000748,-0.000980,0.000905,0.241375
34884,hnusný film,0.102581,-0.001174,-0.062935,-0.001393,-0.001831,-0.034912,-0.137493
34885,hnusný hudba,0.088507,-0.000219,-0.062113,-0.000306,-0.000382,-0.025417,-0.113924
34886,hnusný ten,-0.414053,-0.000577,0.233209,-0.000713,-0.000975,0.183301,0.597354


Podívejme se nyní na slova, která nejvíce do odpadu přispívají.

In [25]:
pd.set_option('display.max_rows', 500)
word_weight.sort_values(by=["goodness"], ascending=True).head(500)

,word,class_0,class_1,class_2,class_3,class_4,class_5,goodness
73665,odpad,12.620505,-0.057804,-5.394984,-0.074172,-0.102210,-6.972376,-19.592881
118400,trapný,5.415656,-0.066532,0.636251,-0.085250,-0.116212,-5.762768,-11.178424
143575,špatný,4.913984,-0.102960,1.790588,-0.131937,-0.183886,-6.252387,-11.166371
104846,sračka,6.460882,-0.034335,-2.568876,-0.044078,-0.017298,-3.785474,-10.246356
0,0,6.960406,-0.020337,-3.989952,-0.025558,-0.035268,-2.882916,-9.843322
69344,nudný,2.617477,-0.074780,4.056421,-0.095789,-0.066858,-6.413056,-9.030534
69140,nuda,1.911530,0.303958,5.050667,-0.100755,-0.136015,-7.004694,-8.916224
106116,stupidní,3.986042,-0.027910,-0.155437,-0.036077,-0.049355,-3.708244,-7.694286
8125,"blbost_,h_^(*3ý)",3.201087,1.239550,0.051522,-0.070373,-0.095297,-4.309413,-7.510500
90457,příšerný,3.729623,-0.031399,0.066867,-0.040648,-0.055386,-3.659002,-7.388626


In [ ]:
Pokud máme jednu jedinou recenzi a chceme ukázat predikované hodnocení, použijeme následující postup.

In [26]:
logist_model.predict(vectorizer.transform(["To byl ten nejlepší film, který jsem kdy viděl."]))

array(['5'], dtype=object)

In [27]:
logist_model.predict(vectorizer.transform(["Že já blbec do toho kina vůbec lezl."]))

array(['0'], dtype=object)

## České filmy

Výše používaný dataset byl poněkud zvláštní - primárně se v něm vyskytovaly recenze s hodnocením odpad, dvě hvězdy a pět hvězd. Zkusme tedy úlohu replikovat na nascrapovaných datech. Konkrétně se budeme věnovat recenzím českých filmů. Ne že bych byl kdovíjak velký fanoušek naší kinematografie, ale recenzí na vyhodnocení bude mnohem méně než dejme tomu při snaze scrapovat sci-fi filmy.
Níže se naléza scrapovací program:
```python
import scrapy
from scrapy.crawler import CrawlerProcess

class ReviewsSpider(scrapy.Spider):
    name = "mega_recenze"
    start_urls = [
         "https://www.csfd.cz/podrobne-vyhledavani/?type%5B%5D=0&genre%5Btype%5D=2&genre%5Binclude%5D%5B%5D=&genre%5Bexclude%5D%5B%5D=&origin%5Btype%5D=2&origin%5Binclude%5D%5B%5D=1&origin%5Bexclude%5D%5B%5D=&year_from=&year_to=&rating_from=&rating_to=&actor=&director=&composer=&screenwriter=&author=&cinematographer=&production=&edit=&sound=&scenography=&mask=&costumes=&tag=&ok=Hledat&_form_=film"
         ]
    
    def parse(self, response):
        films = response.css("table tbody")[0]
        films_core_parts = films.css("tr td a ::attr(href)").extract()
        for part in films_core_parts:
            print("part: ", part)
            film_link = "https://www.csfd.cz" + part + "komentare/"
            print("link: ", film_link)
            yield response.follow (film_link, self.parse_one_film)
        next_page = response.css("div a.button ::attr(href)").extract()[-1]
        if next_page:
            yield scrapy.Request(response.urljoin(next_page), callback=self.parse)        
   
    def parse_one_film(self, response):
        film_name = response.css("title ::text").extract_first()
        film_name = film_name.replace(" | ČSFD.cz", "")
        reviews = response.css("ul.ui-posts-list li")
        for review in reviews:
            yield {
                "film_name":film_name,
                "author":review.css("h5.author a ::text").extract(),
                "stars":review.css("img.rating ::attr(alt)").extract(),
                "word_rating":review.css("strong.rating ::text").extract(),
                "review_text": review.css("p.post ::text").extract(),
            }
        next_film_page = response.css("a.next ::attr(href)").extract_first()
        print(str(next_film_page))
        if next_film_page:
            yield scrapy.Request(response.urljoin(next_film_page), callback=self.parse_one_film)        

if __name__ == "__main__":
    crawler = CrawlerProcess(
        {
            "USER_AGENT": "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
            "FEED_FORMAT": "csv",
            "FEED_URI": f"scrapped_reviews.csv",
            "LOG_LEVEL": "WARN",
        }
    )
    print("Scrapping starts.")
    crawler.crawl(ReviewsSpider)
    crawler.start()
    print("Scrapping ends.")
```

Jakmile máme data na disku, můžeme si je načíst do pandího dataframu a zběžně prohlédnout. Zde je asi na místě poznamenat, že hodnocení "Odpad!" a hvězdy jsou na stránkách umístěné v jiných html tazích. Tím pádem se musely brát informace z obou tagů a pokud tag zorva nebyl přítomen, dostala se nám do dat hodnota NaN.

In [28]:
czech_films_dataset = pd.read_csv("scrapped_reviews.csv", sep=",", header=0)
czech_films_dataset.head()

,film_name,author,stars,word_rating,review_text
0,Pelíšky (1999),golfista,*****,NaN,"Skvělý film se skvělou atmosférou těch ""podiv..."
1,Pelíšky (1999),KevSpa,*****,NaN,Herecký koncert našich nejlepších herců a nez...
2,Pelíšky (1999),Houdini,****,NaN,"Křišťálový glóbus - výběr, Český Lev 1 : Here..."
3,Pelíšky (1999),Galadriel,*****,NaN,"Nádherný film, který ač vypráví o době hodně ..."
4,Pelíšky (1999),Gemini,*****,NaN,"Slabé místo by se určitě našlo, ale nějak si ..."


Nahraďme tedy ve sloupcích "stars" a "word_rating" NaN za prázdný string (""). Následně si vytvořme sloupec "rating" jako slepeninu oněch dvou původních sloupců s hodnoceními.

In [29]:
czech_films_dataset["stars"] = czech_films_dataset["stars"].fillna("")
czech_films_dataset["word_rating"] = czech_films_dataset["word_rating"].fillna("")
czech_films_dataset["rating"] = czech_films_dataset["stars"] + czech_films_dataset["word_rating"]
czech_films_dataset = czech_films_dataset.drop(["stars", "word_rating"], axis=1)
czech_films_dataset.head()

,film_name,author,review_text,rating
0,Pelíšky (1999),golfista,"Skvělý film se skvělou atmosférou těch ""podiv...",*****
1,Pelíšky (1999),KevSpa,Herecký koncert našich nejlepších herců a nez...,*****
2,Pelíšky (1999),Houdini,"Křišťálový glóbus - výběr, Český Lev 1 : Here...",****
3,Pelíšky (1999),Galadriel,"Nádherný film, který ač vypráví o době hodně ...",*****
4,Pelíšky (1999),Gemini,"Slabé místo by se určitě našlo, ale nějak si ...",*****


Jak tedy vlastně dataset vypadá? Počet unikátních jmen rovný 1000 je hodně podezřelý. Nicméně kdyby člověk vlezl na csfd.cz na "podrobné vyhledávání" (úplně nahoře pod hledacím textboxem), pustil by ho interface maximálně na stránku 20. No a na každé stránce je 50 filmů, takže celkový počet recenzovaných filmů rovný tisícovce odpovídá.
Trochu divné je, že nejčastější recenze se vyskytuje 37 a nejedná se o recenzi práznou či jednoslovné zvolání, ale o dosti komplexně vypadající text.

In [30]:
czech_films_dataset.describe()

,film_name,author,review_text,rating
count,197235,197235,197235,197235
unique,1000,21497,196566,7
top,Pelíšky (1999),zette,"Jeden z mých nejoblíbenějších ""večerníčkových...",****
freq,1778,501,37,49164


Jenže když se na situaci podíváme podrobně, zjistíme, že jsme nenarazili na chybu programu, ale na realitu. Opravdu lidé v jeden den (v tomto kroku je stále datum napsání recenze součástí textu recenze) dokáží stejně ohodnotit X filmů. Zejména se jedná o animované pohádky (recenze ve výstupu funkce describe patří Patovi a Matovi) anebo o Kameňáky...

In [31]:
pd.set_option('display.max_rows', 500)
czech_films_dataset[czech_films_dataset["review_text"].duplicated(keep=False)].sort_values(by="review_text").head(500)

,film_name,author,review_text,rating
106914,Tajnosti (2007),DENIROSIS,"***3/4.,(27.7.2007)",***
45149,Marcela (2006),DENIROSIS,"***3/4.,(27.7.2007)",****
153852,Anděl Páně 2 (2016),TuT,",80 %,(28.12.2017)",****
189078,Anděl Páně (2005),TuT,",80 %,(28.12.2017)",****
137499,Knoflíkáři (1997),Gianni_B,",Hudba:, Aleš Březina,(1.9.2010)",
116935,Kráska v nesnázích (2006),Gianni_B,",Hudba:, Aleš Březina,(1.9.2010)",****
179319,Obsluhoval jsem anglického krále (2006),Gianni_B,",Hudba:, Aleš Březina,(1.9.2010)",****
180665,Musíme si pomáhat (2000),Gianni_B,",Hudba:, Aleš Březina,(1.9.2010)",****
166059,Horem pádem (2004),Gianni_B,",Hudba:, Aleš Březina,(1.9.2010)",
35682,Pat a Mat: Trezor (1994),MikeCool,",Krátkometrážní snímek., Každý z dílů Pata a ...",***


Hodnocení alternující mezi hvězdičkami a slovy je poněkud nekonzistentní - pojďme ho konvertovat na čísla či spíše na kategorie reprezentované čísly. To, že tu z čísel -1 až 5 děláme kategorická data znamená, že je nepůjde sčítat, násobit či dělit.

In [32]:
czech_films_dataset["rating"].unique()

array(['*****', '****', '***', 'odpad!', '*', '**', ''], dtype=object)

In [33]:
dict_for_replacing = {"*****":5, "****":4, "***":3, "**":2, "*":1, "odpad!":0, "":-1}
czech_films_dataset["rating"] = czech_films_dataset["rating"].replace(dict_for_replacing)
czech_films_dataset["rating"]= czech_films_dataset["rating"].astype("category")
czech_films_dataset

,film_name,author,review_text,rating
0,Pelíšky (1999),golfista,"Skvělý film se skvělou atmosférou těch ""podiv...",5
1,Pelíšky (1999),KevSpa,Herecký koncert našich nejlepších herců a nez...,5
2,Pelíšky (1999),Houdini,"Křišťálový glóbus - výběr, Český Lev 1 : Here...",4
3,Pelíšky (1999),Galadriel,"Nádherný film, který ač vypráví o době hodně ...",5
4,Pelíšky (1999),Gemini,"Slabé místo by se určitě našlo, ale nějak si ...",5
...,...,...,...,...
197230,Van Helsing (2004),JC_Gucci,"Tento film je buď strašná blbost, nebo geniál...",0
197231,Van Helsing (2004),detrew,Pokud od tohohle snímku neočekáváte víc než j...,3
197232,Van Helsing (2004),SANNY22,"Príbeh, efekty, proste všetko to bolo nechutn...",0
197233,Van Helsing (2004),Gappard,"Sice je to maraz jak po dějové stránce, tak i...",5


Podívejme se na celý text jedné recenze. Informace o datu recenze, tj. ",(24.12.2019)", pro nás nemá význam a tak bychom ji rádi odstranili.

In [34]:
czech_films_dataset["review_text"].iloc[0]

' Skvělý film se skvělou atmosférou těch "podivných" let našeho života. Vždy při jeho sledování obdivuju, jak to všechno mohl s takovou přesností vystihnout režisér, který v té době zrovna přicházel na svět ?! Klobouk dolů pane Hřebejk.,(11.2.2003)'

To půjde nejsnáze s pomocí metody str.replace. Ta má jako první parametr slovo/pattern, který se má nahradit. Druhým  parametrem je pak string, který původní řetězec nahrazuje. Co se týče podoby samotného patternu, "\(", "\)" a "\." reprezentují závorky a tečku - ty mají v rámci regulárních výrazů specifické významy a proto, pokud je člověk opravdu chce použít jako znaky, musí je escapovat zpětným lomítkem. "\d" reprezentuje číslovky. Složené závorky zase říkají, kolikrát se elementy před nimi vyskytující má objevit (to když obsahují jedno číslo), resp. specifikují interval počtu výskytů (když obsahují čísla dvě).

In [35]:
pattern = ",\(\d{1,2}\.\d{1,2}\.\d{4}\)"
czech_films_dataset["review_text"] = czech_films_dataset["review_text"].str.replace(pattern, "")
czech_films_dataset["review_text"].iloc[0]

' Skvělý film se skvělou atmosférou těch "podivných" let našeho života. Vždy při jeho sledování obdivuju, jak to všechno mohl s takovou přesností vystihnout režisér, který v té době zrovna přicházel na svět ?! Klobouk dolů pane Hřebejk.'

Použijme nyní stejný postup jako u předchozích dat. Pouze se musíme omezit na unigramy - ngram_range=(1,3) můj počítač už nezvládal.

In [36]:
czech_films_dataset_strings = [text.strip().lower() for text in czech_films_dataset["review_text"]]

x_train, x_test, y_train, y_test = train_test_split(
     czech_films_dataset_strings, 
     czech_films_dataset["rating"], 
     test_size=0.2, 
     random_state=42
)

In [37]:
vectorizer = TfidfVectorizer(tokenizer = lemmatize_review, ngram_range=(1,1), min_df=10) 
vectorized_train = vectorizer.fit_transform(x_train)
vectorized_test = vectorizer.transform(x_test)

In [38]:
logist_model = LogisticRegression(solver="saga")
logist_model.fit(vectorized_train, y_train)

prediction = logist_model.predict(vectorized_test)

accuracy = sklearn.metrics.accuracy_score(y_test, prediction)
accuracy

0.5158060182016376

In [39]:
conf_matrix = sklearn.metrics.confusion_matrix(y_test, prediction)
conf_matrix

array([[  42,   67,   26,   25,   57,   63,   26],
       [   6, 2150,  692,  371,  251,  223,   77],
       [   8,  826, 1525, 1019,  576,  334,  135],
       [   8,  348,  716, 2380, 1866,  643,  166],
       [   8,  203,  245, 1017, 5530, 2260,  323],
       [   8,  151,  141,  308, 1897, 6146, 1174],
       [   3,  146,   69,  104,  386, 2128, 2574]], dtype=int64)

Vidíme, že přesností už to tak slavné není - pouze cca 52%. Nicméně pohledem na confusion patrix lze nalhédnout, že model zase tak špatný není. Model si často o recenzi za 4* myslí, že je za 5* či za 3*, nicméně odpad jí většinou nepřisuzuje.

In [40]:
ratings_freq = collections.Counter(czech_films_dataset["rating"])
ratings_freq

Counter({5: 27189, 4: 49164, 3: 47657, 0: 18978, 1: 22183, 2: 30497, -1: 1567})

In [41]:
random_with_real_freq = ['-1']*1567 + ['0']*18978 + ['1']*22183 + ['2']*30497 + ['3']*47657 + ['4']*49164 + ['5']*27189
random.shuffle(random_with_real_freq)
fake_prediction = random_with_real_freq[:len(y_test)]

In [42]:
print(sklearn.metrics.accuracy_score(y_test.astype("str"), fake_prediction))
conf_matice = sklearn.metrics.confusion_matrix(y_test.astype("str"), fake_prediction)
conf_matice

0.18807513879382462


array([[   3,   27,   31,   49,   77,   79,   40],
       [  22,  393,  411,  586,  886,  952,  520],
       [  36,  420,  487,  667, 1056, 1121,  636],
       [  41,  575,  730,  974, 1447, 1541,  819],
       [  89,  956, 1077, 1474, 2327, 2332, 1331],
       [  81,  957, 1115, 1512, 2395, 2459, 1306],
       [  50,  524,  623,  832, 1245, 1360,  776]], dtype=int64)

Vidíme tedy, že náš model je o více jak 30% lepší než náhodné tipování.

Pokud bychom chtěli explicitně vidět, jakou pravděpodobnost u každné recenze model přisuzoval jednotlivému počtu hvězd, použijeme predict_proba funkci.

In [49]:
probabilities = logist_model.predict_proba(vectorized_test)
probabilities

array([[0.00153101, 0.00769684, 0.01157824, ..., 0.07110483, 0.57072219,
        0.31023261],
       [0.02381022, 0.15347913, 0.15702001, ..., 0.07533319, 0.33581307,
        0.04711606],
       [0.00235402, 0.35684254, 0.4525543 , ..., 0.02592168, 0.01489128,
        0.02273152],
       ...,
       [0.00294555, 0.02994113, 0.01507784, ..., 0.14994397, 0.20261242,
        0.56048171],
       [0.02079695, 0.03228084, 0.08760327, ..., 0.34719987, 0.26764639,
        0.11584425],
       [0.00312488, 0.00227609, 0.00334997, ..., 0.03140181, 0.79439664,
        0.13808659]])

In [60]:
probab_m1 = [review[0] for review in probabilities]
probab_0 = [review[1] for review in probabilities]
probab_1 = [review[2] for review in probabilities]
probab_2 = [review[3] for review in probabilities]
probab_3 = [review[4] for review in probabilities]
probab_4 = [review[5] for review in probabilities]
probab_5 = [review[6] for review in probabilities]

In [62]:
reviews_probability = pd.DataFrame({
   "text": x_test,
   "real_rating": y_test,
   "predicted_rating": prediction,
   "class_m1": probab_m1,
   "class_0": probab_0,
   "class_1": probab_1,
   "class_2": probab_2,
   "class_3": probab_3,
   "class_4": probab_4,
   "class_5": probab_5
})
reviews_probability

,text,real_rating,predicted_rating,class_m1,class_0,class_1,class_2,class_3,class_4,class_5
153167,"jednoduchá zápletka, skvělá muzika. takovéhle ...",5,4,0.001531,0.007697,0.011578,0.027134,0.071105,0.570722,0.310233
184197,tohle teda gilliamovi fest ujelo. objektivně (...,3,4,0.023810,0.153479,0.157020,0.207428,0.075333,0.335813,0.047116
166315,ani můj komparzistický herecký výkon tenhle fi...,0,1,0.002354,0.356843,0.452554,0.124705,0.025922,0.014891,0.022732
22725,tohle je skvělej film z české kuchyně a kdo ne...,5,5,0.019635,0.083323,0.050432,0.048230,0.040873,0.207131,0.550376
55827,"než jsem si zvykl na poněkud zvláštní animaci,...",3,3,0.001767,0.005831,0.028109,0.093413,0.644231,0.220556,0.006093
...,...,...,...,...,...,...,...,...,...,...
115367,spíše podprůměrný český film se slovenskými he...,2,2,0.000573,0.005602,0.047727,0.934711,0.011032,0.000314,0.000041
109917,"je určitě dobře, že tenhle dokument vzniknul. ...",3,2,0.031797,0.033432,0.175794,0.389594,0.149891,0.087955,0.131537
170841,tento film ve mně zanechal hluboký dojem. byl ...,5,5,0.002946,0.029941,0.015078,0.038997,0.149944,0.202612,0.560482
168719,pachatel se vrací na místo činu?!!?,0,3,0.020797,0.032281,0.087603,0.128628,0.347200,0.267646,0.115844


Když tedy budeme chtt vidět všechny recenze, které měly v reálu pět hvězd, ale model jim dal nulu, napíšeme následující:

In [65]:
mistakes = reviews_probability[(reviews_probability["real_rating"]==5) & (reviews_probability["predicted_rating"]==0)]
mistakes

,text,real_rating,predicted_rating,class_m1,class_0,class_1,class_2,class_3,class_4,class_5
172336,naprosto přesná studie ubohohého xenofobního a...,5,0,0.011483,0.305135,0.117828,0.123253,0.092032,0.194206,0.156062
176760,"naprosto uzasna komedie!!! miluju babovrsky, h...",5,0,0.011954,0.331086,0.132089,0.072676,0.084348,0.098192,0.269655
62911,jak můžete sledovat takový bezduchý brak?proč ...,5,0,0.014502,0.515388,0.229773,0.087983,0.095164,0.033226,0.023964
37746,zeman je zmrd!,5,0,0.037442,0.344298,0.028739,0.057167,0.066086,0.332082,0.134186
98770,mohlo by se točit víc takových filmů místo těc...,5,0,0.008956,0.425689,0.311391,0.138482,0.061845,0.033397,0.020241
190430,"nechapu nektere komentare. ""je to kyc"", ""herci...",5,0,0.021113,0.253674,0.090141,0.242897,0.141704,0.085338,0.165133
83896,"všichni kdo dali známku odpad, film zřejmě nev...",5,0,0.022045,0.638918,0.289026,0.013052,0.018741,0.008200,0.010018
153532,je to dobrota plná ptákovin .... odpady nejs...,5,0,0.022945,0.297172,0.198355,0.099638,0.158341,0.099121,0.124427
25488,všetci čo filmu môj pes killer dali hodnotenie...,5,0,0.031614,0.351047,0.114810,0.120402,0.098712,0.186257,0.097158
25400,lepsie by som to ani ja nenatocil. :o),5,0,0.024170,0.313188,0.097256,0.172327,0.104825,0.202854,0.085380


Občas se holt v recenzi použije tolik negativních slov, že i když je recenze pozitivní, odhad jde do odpadu. Možná by pomohlo použití bi a trigramů (kdyby to počítač zvládl).

In [68]:
mistakes["text"].iloc[4]

'mohlo by se točit víc takových filmů místo těch hovadin co se u nás většinou točí.tenhle dokazuje že to jde a nemusí to skončit jako trapas.'

Dobrá, tahle recenze vypadá velice pozitivně, ale model jí dal nulu. Že by autora podezříval ze sarkasmu? Níže je patrné, že za obrat může slovo "troska", která má takovou váhu, že z recenze 5 udělala recenzi 0 :D. Nicméně když se podíváme na pravděpodobnosti, tak 5* je hned za odpadem...

In [69]:
mistakes["text"].iloc[1]

'naprosto uzasna komedie!!! miluju babovrsky, hraji tam mi oblibei herci a je to mnohem lepsi nez slunce seno nebo kamenak... proste pan troska stvoril o5 neco uchvatnyho... tesim se na dvojku! :) tolik srandy jsem jeste nezazila...'

In [81]:
word_weight = pd.DataFrame({
    "word":sorted(vectorizer.vocabulary_, key=lambda x: vectorizer.vocabulary_[x]),
    "class_0": logist_model.coef_[1],
    "class_1": logist_model.coef_[2],
    "class_2": logist_model.coef_[3],
    "class_3": logist_model.coef_[4],
    "class_4": logist_model.coef_[5],
    "class_5":logist_model.coef_[6],
    "goodness": logist_model.coef_[6]-logist_model.coef_[1]
})

In [72]:
word_weight[word_weight["word"].str.contains("troska")]

,word,class_0,class_1,class_2,class_3,class_4,class_5,goodness
24747,troska,1.391303,-0.864848,-0.394164,0.068865,-0.188485,-0.592356,-1.98366


Bez slova "troska"

In [79]:
logist_model.predict(vectorizer.transform(["naprosto uzasna komedie!!! miluju babovrsky, hraji tam mi oblibei herci a je to mnohem lepsi nez slunce seno nebo kamenak...proste pan "]))

array([5], dtype=int64)

Se slovek "troska"

In [80]:
logist_model.predict(vectorizer.transform(["naprosto uzasna komedie!!! miluju babovrsky, hraji tam mi oblibei herci a je to mnohem lepsi nez slunce seno nebo kamenak...proste pan troska "]))

array([0], dtype=int64)